In [2]:
import os
import re
import json
import codecs
import locale
import requests
import datetime

import tabula

import pandas as pd
locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')


'it_IT.UTF-8'

In [3]:
#----------------------------------------------------------------
# Configurations section
#----------------------------------------------------------------

# Population data.
regions_data_file_path = os.path.join("..","data", "Istat", "Tavole-Bilancio-demografico-2018.xlsx")

# Url of the pdf file to download:
url_region_pdf = "http://www.salute.gov.it/imgs/C_17_notizie_4820_0_file.pdf"
pdf_file_name = os.path.join(os.sep, "tmp", "temp_data_file.pdf")


In [42]:
regions_df = pd.read_excel(regions_data_file_path, sheet_name=0)
regions_df.drop(regions_df.index[:5], inplace=True)
regions_df.drop(regions_df.index[regions_df.shape[0] - 2:regions_df.shape[0]], inplace=True)
regions_df.rename(columns={regions_df.columns[0]: "Regione"
                          ,regions_df.columns[1]: "Popolazione"}
                 ,inplace = True)
regions_df = regions_df.loc[:,["Regione","Popolazione"]]
regions_df["Popolazione"] = pd.to_numeric(regions_df['Popolazione'])
regions_df.sort_values(by=["Regione"], inplace=True)

(regions_df.shape,regions_df.dtypes)

((28, 2),
 Regione        object
 Popolazione     int64
 dtype: object)

In [43]:
regions_df.head(30)

,Regione,Popolazione
19,Abruzzo,1315196
23,Basilicata,567118
9,Bolzano-Bozen,527750
24,Calabria,1956687
21,Campania,5826860
29,Centro,12050054
14,Emilia-Romagna,4452629
12,Friuli-Venezia Giulia,1216853
31,Isole,6675165
32,Italia,60483973


In [107]:
#----------------------------------------------------------------
#
#----------------------------------------------------------------

def get_web_file(url):
    """
    
    :param url: 
    :return (rv, content):
    """
    ok_statuses = [200, 201, 202]
    rv = False
    result = None
    try:
        result = requests.get(url_region_pdf)
        if result.status_code not in ok_statuses:
            print("Get data failed. Received error code: {er}".format(er=str(result.status_code)))
        else:
            result = result.content
    except Exception as ex:
        print("get_web_file failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, result)    
        
def save_content_to_file(file_name, content):
    """
    
    :param file_name: 
    :return rv:
    """
    rv = False
    try:
        with open(file_name, "wb") as fh:
            fh.write(content)
    except Exception as ex:
        print("save_content_to_file failed - {ex}".format(ex=ex))
    else:
        rv = True
    return rv
   
def pdf_to_dataframe(pdf_file_name, version = "v2"):
    """
    
    :param version: valid values v1 or v2; 
    :param pdf_file_name: 
    :return rv:
    """
    rv = False
    df = None
    try:
        df = tabula.read_pdf(pdf_file_name, pages='all')
        #print("Df list len: {l}".format(l=len(df)))
        
        if version == "v2":
            csv_file = os.path.splitext(pdf_file_name)[0] + ".csv"
            tabula.convert_into(pdf_file_name, csv_file, output_format="csv", pages='all')
            list_reg = [] 
            with open(csv_file, "r") as fh:
                start = False
                end = False
                reg = re.compile("(\d{1,3}) (\d)")
                for line in fh:
                    if line.startswith("Lombardia") == True:
                        start = True
                    if line.startswith("TOTALE") == True:
                        end = True
                        start = False
                    if start == True:
                        line = line.replace(".", "")
                        line = line.replace("+ ", "")
                        #line = line.replace(" ", ",")
                        line = reg.sub("\\1,\\2", line)
                        line = line.replace("\n", "")
                        list_reg.append(line)
            #print("{lst}".format(lst=list_reg))
            df[0] = pd.DataFrame([line.split(",") for line in list_reg])

    except Exception as ex:
        print("pdf_to_dataframe failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, df)

def get_row_from_Italy_df(df):
    """
    :param df: 
    :return (rv, content):
    """
    rv = False
    csv_row = []
    try:
        csv_row.append(sample_date)
        
        infected_label = df.columns.values[0]
        infected_num = locale.atoi(df.columns.values[1])
        csv_row.append(str(infected_num))
        
        df.set_index(["ATTUALMENTE POSITIVI"], inplace=True)
        df["Totals"] = df[df.columns[0]].apply(lambda row: int(row) if row.is_integer() else int(row * 1000) )
        
        tot_deads = df.loc["TOTALE DECEDUTI", ['Totals']]
        tot_deads = str(int(tot_deads.values[0]))
        csv_row.append(tot_deads)
        
        tot_recovered = df.loc["TOTALE GUARITI", ['Totals']]
        tot_recovered = str(int(tot_recovered.values[0]))
        csv_row.append(tot_recovered)
        
        tot_infected = df.loc["CASI TOTALI", ['Totals']]
        tot_infected = str(int(tot_infected.values[0]))
        csv_row.append(tot_infected)
        
        csv_row = ",".join(csv_row)
        
    except Exception as ex:
        print("get_row_from_Italy_df failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, csv_row)

def append_row(file_name, row):
    """
    
    :param file_name:
    :param row:
    """
    rv = False
    try:
        with open(file_name, 'a') as fh:
            row = row + "\n"
            fh.write(row)
    except Exception as ex:
        print("append_row failed - {ex}".format(ex=ex))
    else:
        rv = True
    return rv

def refactor_region_df(df, version="v2"):
    """
    
    :param df: a list vor v2 or a dataframe for v1
    :param version: valid values v1 or v2; 
    :return (rv, df_region):
    """
    rv = False
    df_res = None
    try:
        if version == "v1":
            df_res = df.drop([0,1])
            df_res.rename(columns={df.columns[0]: "Regione"
                                  ,df.columns[1]: "Ricoverati con sintomi"
                                  ,df.columns[2]: "Terapia intensiva"
                                  ,df.columns[3]: "Isolamento domiciliare"
                                  ,df.columns[4]: "Totale attualmente positivi"
                                  ,df.columns[5]: "DIMESSI/GUARITI"
                                  ,df.columns[6]: "DECEDUTI"
                                  ,df.columns[7]: "CASI TOTALI"
                                  ,df.columns[8]: "TAMPONI"
                                  },
                         inplace = True)

            for idx in range(1, 6):
                df_res[df_res.columns[idx]] = df_res[df_res.columns[idx]].apply(lambda row: row.replace(".", "") )
            for idx in range(6, 9):
                df_res[df_res.columns[idx]] = df_res[df_res.columns[idx]].apply(lambda row: str(int(row)) if row.is_integer() else str(int(row * 1000)) )

        elif version == "v2":
            df_res = df
            df_res.rename(columns={df_res.columns[0]: "Regione"
                                  ,df_res.columns[1]: "Ricoverati con sintomi"
                                  ,df_res.columns[2]: "Terapia intensiva"
                                  ,df_res.columns[3]: "Isolamento domiciliare"
                                  ,df_res.columns[4]: "Totale attualmente positivi"
                                  ,df_res.columns[5]: "DIMESSI/GUARITI"
                                  ,df_res.columns[6]: "DECEDUTI"
                                  ,df_res.columns[7]: "CASI TOTALI"
                                  ,df_res.columns[8]: "INCREMENTO CASI TOTALI"
                                  ,df_res.columns[9]: "TAMPONI"
                                  ,df_res.columns[10]: "CASI TESTATI" 
                                  ,df_res.columns[11]: "INCREMENTO TAMPONI" 
                              },
                          inplace = True)
            df_res["Totale attualmente positivi"] = pd.to_numeric(df_res["Totale attualmente positivi"])
            df_res["INCREMENTO CASI TOTALI"] = pd.to_numeric(df_res["INCREMENTO CASI TOTALI"])
            df_res["CASI TOTALI"] = pd.to_numeric(df_res["CASI TOTALI"])
            
        else:
            msg = "Unknown \"${v}\" version number.".format(v=version)
            print(msg)
            raise Exception(msg)

    except Exception as ex:
        print("refactor_region_df failed - {ex}".format(ex=ex))
    else:
        rv = True  
        df_res.set_index("Regione", inplace=True)
    print("rv -> {rv}".format(rv=rv))
    return (rv, df_res)

def get_region_row(df, region_name):
    """
    
    :param df: 
    :param region_name: 
    :return (rv, row):
    """
    rv = False
    row = None
    try:
        csv_row = []
        csv_row.append(sample_date)
        
        value = df.loc[region_name]['Ricoverati con sintomi']
        csv_row.append(value)
        
        value = df.loc[region_name]['Terapia intensiva']
        csv_row.append(value)

        value = df.loc[region_name]['Isolamento domiciliare']
        csv_row.append(value)

        value = df.loc[region_name]['Totale attualmente positivi']
        csv_row.append(value)

        value = df.loc[region_name]['DIMESSI/GUARITI']
        csv_row.append(value)
        
        value = df.loc[region_name]['DECEDUTI']
        csv_row.append(value)
        
        value = df.loc[region_name]['CASI TOTALI']
        csv_row.append(value)

        value = df.loc[region_name]['TAMPONI']
        csv_row.append(value)        
        
        row = ",".join(csv_row)
        
    except Exception as ex:
        print("get_region_row failed - {ex}".format(ex=ex))
    else:
        rv = True  
    return (rv, row)


In [108]:
df_covid_regions = None

rv = False
result = get_web_file(url_region_pdf)
if result[0] == True:
    rv = save_content_to_file(pdf_file_name, result[1])
else:
    assert False, "File download failure."
if rv == True:
    df_list = pdf_to_dataframe(pdf_file_name)
if df_list[0] == True:
    df_covid_regions = df_list[1][0]
    df_covid_regions = refactor_region_df(df_covid_regions, version="v2")
    if df_covid_regions[0] == True:
        df_covid_regions = df_covid_regions[1]
        df_covid_regions.sort_index(inplace=True)
    else:
        assert False, "Refactor dataframe."
else:
    assert False, "Unable to transform the downloaded file to dataframe."


Got stderr: May 28, 2020 2:18:53 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F1 are not implemented in PDFBox and will be ignored
May 28, 2020 2:18:53 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F2 are not implemented in PDFBox and will be ignored
May 28, 2020 2:18:53 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F3 are not implemented in PDFBox and will be ignored
May 28, 2020 2:18:54 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F1 are not implemented in PDFBox and will be ignored
May 28, 2020 2:18:54 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F2 are not implemented in PDFBox and will be ignored
May 28, 2020 2:18:54 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenT

rv -> True


In [100]:
df_covid_regions.dtypes

Ricoverati con sintomi         object
Terapia intensiva              object
Isolamento domiciliare         object
Totale attualmente positivi     int64
DIMESSI/GUARITI                object
DECEDUTI                       object
CASI TOTALI                    object
INCREMENTO CASI TOTALI          int64
TAMPONI                        object
CASI TESTATI                   object
INCREMENTO TAMPONI             object
dtype: object

In [110]:
df_covid_regions.rename(index={
    "Valle d'Aosta" : "Valle d'Aosta-Vallée d'Aoste"
   ,"Emilia Romagna": "Emilia-Romagna"
   ,"Friuli VG"     : "Friuli-Venezia Giulia"
},inplace=True)

# how{‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner’
merged = pd.merge(df_covid_regions
                ,regions_df
                ,left_index = True
                ,right_on = 'Regione'
                ,how = "inner")
merged["100k"] = merged["Popolazione"].apply(lambda row: row / 100000) 
merged["Infected_4_100k"] = merged.apply(lambda row: row["Totale attualmente positivi"] / row["100k"], axis=1) 
merged["Infected_4_100k_tot"] = merged.apply(lambda row: row["CASI TOTALI"] / row["100k"], axis=1) 

merged[["Regione", "Popolazione", "Totale attualmente positivi", "CASI TOTALI","100k", "Infected_4_100k", "Infected_4_100k_tot"]]

,Regione,Popolazione,Totale attualmente positivi,CASI TOTALI,100k,Infected_4_100k,Infected_4_100k_tot
19,Abruzzo,1315196,866,3235,13.15196,65.845699,245.970943
23,Basilicata,567118,36,399,5.67118,6.347885,70.355728
24,Calabria,1956687,190,1158,19.56687,9.710291,59.181668
21,Campania,5826860,1146,4773,58.26860,19.667540,81.913758
14,Emilia-Romagna,4452629,3998,27627,44.52629,89.789650,620.464898
12,Friuli-Venezia Giulia,1216853,356,3255,12.16853,29.255793,267.493280
18,Lazio,5896693,3488,7672,58.96693,59.151799,130.106824
13,Liguria,1556981,1269,9589,15.56981,81.503885,615.871356
7,Lombardia,10036258,24037,87801,100.36258,239.501615,874.838012
17,Marche,1531753,1450,6718,15.31753,94.662782,438.582461


In [102]:

#merged[["Regione", "Popolazione", "Totale attualmente positivi", "100k"]].apply(lambda row : print(type(row)))

In [90]:
merged.columns

Index(['Ricoverati con sintomi', 'Terapia intensiva', 'Isolamento domiciliare',
       'Totale attualmente positivi', 'DIMESSI/GUARITI', 'DECEDUTI',
       'CASI TOTALI', 'INCREMENTO CASI TOTALI', 'TAMPONI', 'CASI TESTATI',
       'INCREMENTO TAMPONI', 'Regione', 'Popolazione', '100k',
       'Infected_4_100k'],
      dtype='object')